Problem 1-6
======================

In [2]:
import numpy as np
import collections

In [26]:
class simplex(object):
    
    def __init__(self, c, A, b):
        self.c = c
        self.A = A
        self.b = b
        self.m = len(self.b)
        
        #Instantiate index list
        self.index = np.arange(0, len(self.c) + len(self.b))
        self.n = len(self.index) 
        self.x = np.ones(len(self.index))
        
        for i in range (0, len(self.c)):
            self.x[i] = 0
        
        self.show0 = np.where(self.x == 0)
        self.show1 = np.where(self.x != 0)
        self.bi = self.index[[self.show0]]
        self.nbi = self.index[[self.show1]]
        self.index = np.append(self.nbi, self.bi)
        
        
        #self.check = self.A @ np.zeros(len(self.c))
        
        #for i in range (0, len(self.b)):
            #if self.check[i] > self.b[i]:
                #raise ValueError("Problem not feasible at origin")
        
        self.h = 0
        self.h1 = 0
        self.prow = 1
        self.pcol = 1
        self.enter = 0
        self.leave = 0
        self.Tratio = -1
        self.Tratiomin = 50000
        self.finalx = np.zeros((self.m + self.n))
        
        #Instantiate dictionaries 
        self.bsolved = {}
        self.nbsolved = {}
        self.bsolved1 = {}
        self.nbsolved1 = {}
        
    def update(self, ):
        self.basic = self.index[:len(self.b)]
        self.nonbasic = self.index[len(self.b):]

    def tableau(self):
        #Instantiate tableau
        self.tab = np.array((self.m + 1, 2 + self.n + self.n))
        self.zero = np.array((0))
        self.block1 = np.hstack((self.zero, self.b)).reshape(self.m + 1,1)
        self.zerov1 = np.zeros((self.m))
        self.cbar = - np.hstack((self.c, self.zerov1))
        self.Abar = np.hstack ((self.A, np.eye((self.m))))
        self.block2 = np.vstack((self.cbar, self.Abar))
        self.one = np.array((1))
        self.zerov2 = np.zeros((self.m))
        self.block3 = np.hstack((self.one, self.zerov2)).reshape(self.m + 1,1)
        self.tab = np.hstack((self.block1, self.block2, self.block3))
        
        
    def pivot(self):
        
        for i in range (1, np.shape(self.tab)[1]):
            if self.tab[0][i] < 0:
                self.pcol = i 
                break
        self.enter = i - 1
        
        if np.all(self.tab[1:,self.pcol] < 0):
            raise ValueError ("No solution")
        
        for j in range(1, np.shape(self.tab)[0]):
            
            if self.tab[j, self.pcol] > 0:
                self.Tratio = float(self.tab[j][0] / self.tab[j][self.pcol])
                
                if self.Tratiomin > self.Tratio: 
                    self.Tratiomin = self.Tratio
                    self.Tmin_index = j
                    self.prow = j 
                    self.leave = self.index[j - 1]
                
                #check this if necessary
                elif self.Tratiomin == self.Tratio:
                    if self.leave > self.index[j]:
                        self.prow = j 
                        self.leave = self.index[j - 1]

        #self.prow +=1
        #self.pcol +=1
    
    def rowop(self):
        
        #Check boundedness in previous method
        
        self.tab[self.prow][:]  = self.tab[self.prow][:] / self.tab[self.prow][self.pcol] + 0.001
        
        for i in range (0, np.shape(self.tab)[0]):
            
            if i != self.prow:
                
                self.tab[i][:] =  self.tab[i][:] - (self.tab[i][self.pcol]) * self.tab[self.prow][:]
        
    def solve(self):
        
        self.tableau()
        
        while np.any(self.tab[0][:] < 0 ):
            self.pivot()
            self.h = np.where(self.index == self.enter)
            self.h1 = np.where(self.index == self.leave)
            self.index[self.h1], self.index[self.h] = self.index[self.h], self.index[self.h1]
            self.rowop()
        
        print(self.index)
        
        #Make dict
        self.bsolved = {}
        self.nbsolved = {}
        
        for i in range (0, self.m):
            self.bsolved1 = {self.index[i] : self.tab[i+1][0]}
            self.bsolved = {**self.bsolved, **self.bsolved1}
        
        for j in range (i + 1, self.n):
            self.nbsolved1 = {self.index[j] : 0}
            self.nbsolved = {**self.nbsolved, **self.nbsolved1}        
        
        self.bsolved = collections.OrderedDict(sorted((self.bsolved).items()))
        
        self.nbsolved = collections.OrderedDict(sorted((self.nbsolved).items()))
        
        return self.tab[0][0], self.bsolved, self.nbsolved
        
#Test functions
        
c = np.array([3, 2])
A = np.array(([1, -1], [3, 1], [4,3]))
b = np.array([2, 5, 7])

calc = simplex(c, A, b)
print(calc.solve())

[2 0 1 3 4]
(5.2014781547714515, OrderedDict([(0, 1.601109130981021), (1, 0.19907538091419413), (2, 0.59796624993317304)]), OrderedDict([(3, 0), (4, 0)]))


Problem 7
============

In [27]:
#filename = '/Users/bryanchia/Desktop/BootCamp2017/ProbSets/Comp/Wk4_DifIntOpt/productMix.npz'
filename = '/Users/bryanchia/Desktop/productMix.npz'

data = np.load(filename)
A = data['A']
p = data['p']
m = data['m']
d = data['d']

A = np.vstack((A, np.eye(A.shape[1])))
b = np.concatenate((m,d))
prodmix = simplex(p, A, b)

print(prodmix.solve())

[ 4  2  6  5  8  9 10  1  7  0  3]
(7490.5469132727012, OrderedDict([(2, 9.9711509840280232), (4, -0.16362891875814967), (5, 7.7142119667842053), (6, 1.0172076604646918), (8, 12.285788033215795), (9, -0.1425393308600178), (10, 10.0)]), OrderedDict([(0, 0), (1, 0), (3, 0), (7, 0)]))
